In [1]:

import pandas as pd
import networkx as nx
import os
import copy


In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [3]:
from ema_workbench import (
    Policy,
    ema_logging,
    MultiprocessingEvaluator,
    save_results, 
    load_results,
    Samplers
)
from problem_formulation import get_model_for_problem_formulation



/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")


In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)


<Logger EMA (DEBUG)>

### EPA141 Final Assignment: Exploratory Analysis Setup

This script configures and runs EMA Workbench experiments to explore the combined impact of external uncertainties and individual policy levers on model outcomes. We create a new function, named run_exploratory_experiments to transform levers into inputs and sample them in a similar fashion to the uncertainities, using Latin Hypercube. It is designed for an initial exploratory analysis phase to identify key drivers of the desired outcomes and to understand the space of possibilities. Literature supporting this approach (number of scenarios, exploring levers as factors):
- Bankes, S. (1993). Exploratory Modeling for Policy Analysis. Operations Research, 41(3), 435-449.
 - Bryant, B. P., & Lempert, R. J. (2010). Thinking inside the box: A participatory, computer-assisted approach to scenario discovery. Technological Forecasting & Social Change, 77(1), 34-49.
- Walker, W. E., Marchau, V. A. W. J., & Kwakkel, J. H. (2013). Uncertainty in the framework of Policy Analysis. In W. A. H. Thissen & W. E. Walker (Eds.), Public Policy Analysis:New Developments (pp. 215-261). Springer.
- Moallemi, E. A., Kwakkel, J., de Haan, F. J., & Bryan, B. A. (2020). Exploratory modeling for analyzing coupled human-natural systems under uncertainty. Global Environmental Change, 65, 102186.
- Course Material: "Final_Case_Study_Modeling_Steps.docx" (recommends 1000-3000 simulations).

This code is used to generate exploratory experimental data for different problem forumlations of interest to our clients (Group 7 and 21). Every time the problem forumation is changed, a new name should be given for the variable `filename` to ensure you avoid overwriting previous data. 

In [5]:
def run_exploratory_experiments(problem_formulation_id, num_scenarios, output_filename, sampling_method=None, active_rfr_dikes=None):
    """
    Sets up and runs exploratory experiments where both uncertainties and individual
    policy levers are sampled. Optionally turns RfR levers ON for specific dike rings.
    
    Args:
        problem_formulation_id (int): ID for the problem formulation to load.
        num_scenarios (int): Number of scenarios (experimental runs) to perform.
        output_filename (str): Filename for saving the results (e.g., "results.tar.gz").
        sampling_method (str, optional): Name of sampling method (e.g., Samplers.SOBOL).
        active_rfr_dikes (list of str, optional): List of dike names (e.g., ["A.5"]) to activate RfR for.
    """
    ema_logging.log_to_stderr(ema_logging.INFO)

    # --- 1. Load the Base Model Definition ---
    try:
        from problem_formulation import get_model_for_problem_formulation
        dike_model, _ = get_model_for_problem_formulation(problem_formulation_id)
        print(f"INFO: Loaded model for problem formulation {problem_formulation_id}.")

        # Force RfR levers ON for selected dikes
        if active_rfr_dikes:
            modified_levers = []
            for lev in dike_model.levers:
                if "RfR" in lev.name and any(dike in lev.name for dike in active_rfr_dikes):
                    modified_levers.append(IntegerParameter(lev.name, 1, 1))  # Always ON
                else:
                    modified_levers.append(lev)
            dike_model.levers = modified_levers
            print(f"INFO: RfR levers activated for dike(s): {active_rfr_dikes}")

    except Exception as e:
        print(f"ERROR: Could not load model from problem_formulation.py: {e}")
        return

    # --- 2. Prepare Model for Exploratory Lever Sampling ---
    exploratory_factors = []

    if hasattr(dike_model, 'uncertainties'):
        exploratory_factors.extend(copy.deepcopy(dike_model.uncertainties))
    else:
        print("WARNING: dike_model has no 'uncertainties' attribute.")

    original_levers = []
    if hasattr(dike_model, 'levers') and dike_model.levers:
        original_levers = copy.deepcopy(dike_model.levers)
        exploratory_factors.extend(original_levers)
        print(f"INFO: {len(original_levers)} policy levers will be sampled as factors.")
    else:
        print("WARNING: dike_model has no 'levers' or an empty list of levers.")

    if not exploratory_factors:
        print("ERROR: No uncertainties or levers found to explore. Aborting.")
        return

    dike_model.uncertainties = exploratory_factors
    dike_model.levers = []

    print(f"INFO: Total number of factors to be sampled: {len(dike_model.uncertainties)}")

    # --- 3. Run Experiments ---
    print(f"INFO: Starting exploratory experiments with {num_scenarios} scenarios...")

    try:
        with MultiprocessingEvaluator(dike_model) as evaluator:
            if sampling_method is None:
                results_tuple = evaluator.perform_experiments(
                    scenarios=num_scenarios,
                    policies=None,
                )
            else:
                results_tuple = evaluator.perform_experiments(
                    scenarios=num_scenarios,
                    policies=None,
                    uncertainty_method=sampling_method
                )
        print(f"INFO: Experiment run completed. {len(results_tuple[0])} scenarios were executed.")
        experiments, outcomes = results_tuple
    except Exception as e:
        print(f"ERROR: An error occurred during perform_experiments: {e}")
        return

    # --- 4. Save Results ---
    output_dir = "../experimental data"
    file_path = os.path.join(output_dir, output_filename)

    if not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir)
            print(f"INFO: Created output directory: {output_dir}")
        except OSError as e:
            print(f"ERROR: Could not create directory {output_dir}: {e}")
            return

    try:
        save_results(results_tuple, file_path)
        print(f"INFO: Exploratory results successfully saved to: {file_path}")
    except Exception as e:
        print(f"ERROR: Failed to save results to {file_path}: {e}")

In [8]:
#execute the function 
# Define experiment parameters
PROBLEM_FORMULATION_ID = 5 #helps our clients and to test the priorities of other actors
# Number of scenarios (runs), aiming for 1000-3000 as per literature and guidelines
# due to the function, the arrays get super messed up with around 2000 runs. 1000 is optimal. 
NUMBER_OF_SCENARIOS = 2000 
OUTPUT_FILENAME = "pf_5_exploratory_runs.tar.gz"
SAMPLING_METHOD = None
active_rfr_dikes = ["A.1", "A.4", "A.3"]  # dikes RWS wants to activate RfR for


run_exploratory_experiments(PROBLEM_FORMULATION_ID, NUMBER_OF_SCENARIOS, OUTPUT_FILENAME, SAMPLING_METHOD,  active_rfr_dikes=None)


INFO: Loaded model for problem formulation 5.
INFO: 31 policy levers will be sampled as factors.
INFO: Total number of factors to be sampled: 50
INFO: Starting exploratory experiments with 2000 scenarios...


/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed -

INFO: Experiment run completed. 2000 scenarios were executed.


[MainProcess/INFO] results saved successfully to /Users/precupada/decision_making_assignments/final assignment EPA141/experimental data/pf_5_exploratory_runs.tar.gz


INFO: Exploratory results successfully saved to: ../experimental data/pf_5_exploratory_runs.tar.gz
